In [1]:
import setup
setup.init_django()

In [2]:
import helpers.clients as helper_clients

In [3]:
company_name = "Google"
company_ticker = "GOOG"
multiplier = 1
from_date = "2024-01-09"
to_date = "2025-01-09"



In [4]:
client = helper_clients.PolygonAPIClient(
    ticker=company_ticker,
    multiplier=multiplier,
    from_date=from_date,
    to_date=to_date
    
)
dataset = client.get_stock_data()

In [5]:
len(dataset)

5000

In [6]:
new_stock_data = dataset[0]
new_stock_data

{'open_price': 140.41,
 'close_price': 140.41,
 'high_price': 140.41,
 'low_price': 140.41,
 'number_of_trades': 3,
 'volume': 134,
 'volume_weighted_average': 140.3984,
 'time': datetime.datetime(2024, 1, 9, 9, 2, tzinfo=<UTC>)}

In [7]:
from market.models import Company, StockQuote

In [8]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)

In [9]:
company_obj

<Company: Company object (3)>

In [10]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []

    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [11]:
StockQuote.objects.all().count()

10000